In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [9]:
%cd /content/gdrive/MyDrive/mutsa_2nd_project

/content/gdrive/MyDrive/mutsa_2nd_project


In [ ]:
import os
import shutil
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm

# 경로 설정
BASE_DIR = "/content/gdrive/MyDrive/mutsa_2nd_project/MVTec_data"
OUTPUT_DIR = "/content/gdrive/MyDrive/mutsa_2nd_project/mutsa_yolo_dataset3"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 결합 및 제외 규칙 정의
merge_rules = {
    "bottle": {"broken": ["broken_large", "broken_small"]},
    "screw": {"thread": ["thread_side", "thread_top"], "scratch": ["scratch_neck", "scratch_head"]},
    "pill": {"crack": ["crack", "contamination"]},
}
exclude_rules = {"pill": ["combined"]}

# YOLO 형식 클래스 매핑 (최종 클래스 이름 지정)
final_classes = [
    "bottle_broken", "bottle_contamination", "capsule_crack", "capsule_faulty_imprint",
    "capsule_poke", "capsule_scratch", "capsule_squeeze", "pill_color", "pill_crack",
    "pill_faulty_imprint", "pill_pill_type", "pill_scratch", "screw_manipulated_front",
    "screw_thread", "screw_scratch", "tile_crack", "tile_glue_strip", "tile_gray_stroke",
    "tile_oil", "tile_rough", "toothbrush_defective"
]

# 클래스 이름 매핑 생성
class_name_to_index = {name: idx for idx, name in enumerate(final_classes)}

# 파일 이동 및 YOLO 세그멘테이션 형식 데이터 생성
def move_and_label_files(base_dir, output_dir, merge_rules, exclude_rules, class_name_to_index):
    images_dir = os.path.join(output_dir, "images")
    labels_dir = os.path.join(output_dir, "labels")
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)

    for obj_class in tqdm(os.listdir(base_dir)):
        obj_path = os.path.join(base_dir, obj_class)
        if not os.path.isdir(obj_path):
            continue

        test_path = os.path.join(obj_path, "test")
        ground_truth_path = os.path.join(obj_path, "ground_truth")

        # 처리할 결함 클래스 가져오기
        defect_folders = os.listdir(test_path)
        if obj_class in exclude_rules:
            defect_folders = [f for f in defect_folders if f not in exclude_rules[obj_class]]

        for new_defect, merge_folders in merge_rules.get(obj_class, {}).items():
            defect_folders = [f for f in defect_folders if f not in merge_folders]
            defect_folders.append(new_defect)
            for folder in merge_folders:
                folder_path = os.path.join(test_path, folder)
                if os.path.exists(folder_path):
                    shutil.move(folder_path, os.path.join(test_path, new_defect))
                folder_path = os.path.join(ground_truth_path, folder)
                if os.path.exists(folder_path):
                    shutil.move(folder_path, os.path.join(ground_truth_path, new_defect))

        # YOLO 데이터셋 생성
        for defect in defect_folders:
            defect_name = f"{obj_class}_{defect}"
            defect_index = class_name_to_index[defect_name]
            defect_test_path = os.path.join(test_path, defect)
            defect_gt_path = os.path.join(ground_truth_path, defect)

            if not os.path.exists(defect_test_path) or not os.path.exists(defect_gt_path):
                continue

            test_images = glob(os.path.join(defect_test_path, "*.png"))
            for image_path in test_images:
                base_name = os.path.basename(image_path)
                gt_path = os.path.join(defect_gt_path, base_name.replace(".png", "_mask.png"))

                if not os.path.exists(gt_path):
                    continue

                # 이미지 복사
                new_image_path = os.path.join(images_dir, f"{obj_class}_{defect}_{base_name}")
                shutil.copy(image_path, new_image_path)

                # 레이블 생성
                label_path = os.path.join(labels_dir, f"{obj_class}_{defect}_{base_name.replace('.png', '.txt')}")
                with open(label_path, "w") as f:
                    mask = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)
                    height, width = mask.shape

                    # 바운딩 박스 좌표
                    y_indices, x_indices = mask.nonzero()
                    if len(x_indices) == 0 or len(y_indices) == 0:
                        continue

                    x_min, x_max = x_indices.min(), x_indices.max()
                    y_min, y_max = y_indices.min(), y_indices.max()

                    # 바운딩 박스 중심, 크기 (YOLO 형식)
                    x_center = (x_min + x_max) / 2 / width
                    y_center = (y_min + y_max) / 2 / height
                    bbox_width = (x_max - x_min) / width
                    bbox_height = (y_max - y_min) / height

                    # 폴리곤 좌표 (YOLOv8 세그멘테이션 형식)
                    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    segmentation = []
                    for contour in contours:
                        for point in contour:
                            x, y = point[0]
                            # 소수점 5자리로 반올림하여 저장
                            segmentation.append(f"{round(x / width, 5)},{round(y / height, 5)}")

                    # 소수점 5자리로 반올림하여 바운딩 박스 좌표 저장
                    x_center = round(x_center, 5)
                    y_center = round(y_center, 5)
                    bbox_width = round(bbox_width, 5)
                    bbox_height = round(bbox_height, 5)

                    segmentation_str = " ".join(segmentation)
                    f.write(f"{defect_index} {x_center} {y_center} {bbox_width} {bbox_height} {segmentation_str}\n")

# 실행
move_and_label_files(BASE_DIR, OUTPUT_DIR, merge_rules, exclude_rules, class_name_to_index)

100%|██████████| 12/12 [07:39<00:00, 38.27s/it]


In [ ]:
import os
import shutil
from glob import glob
from tqdm import tqdm
import random

# 경로 설정
BASE_DIR = "/content/gdrive/MyDrive/mutsa_2nd_project/mutsa_yolo_dataset3"
OUTPUT_SPLIT_DIR = "/content/gdrive/MyDrive/mutsa_2nd_project/3_split_data"
os.makedirs(OUTPUT_SPLIT_DIR, exist_ok=True)

# train, val, test 폴더 생성
train_images_dir = os.path.join(OUTPUT_SPLIT_DIR, "train", "images")
train_labels_dir = os.path.join(OUTPUT_SPLIT_DIR, "train", "labels")
val_images_dir = os.path.join(OUTPUT_SPLIT_DIR, "val", "images")
val_labels_dir = os.path.join(OUTPUT_SPLIT_DIR, "val", "labels")
test_images_dir = os.path.join(OUTPUT_SPLIT_DIR, "test", "images")
test_labels_dir = os.path.join(OUTPUT_SPLIT_DIR, "test", "labels")

os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)
os.makedirs(test_images_dir, exist_ok=True)
os.makedirs(test_labels_dir, exist_ok=True)

# 데이터셋 분할 비율 (train, val, test)
TRAIN_RATIO = 0.7
VAL_RATIO = 0.15
TEST_RATIO = 0.15

# 이미지 및 레이블 이동 함수
def move_images_and_labels(src_images_dir, src_labels_dir, dst_images_dir, dst_labels_dir, file_list):
    for file_name in file_list:
        # 이미지 파일 복사
        src_image_path = os.path.join(src_images_dir, file_name)
        dst_image_path = os.path.join(dst_images_dir, file_name)
        shutil.copy(src_image_path, dst_image_path)

        # 레이블 파일 복사
        src_label_path = os.path.join(src_labels_dir, file_name.replace(".png", ".txt"))
        dst_label_path = os.path.join(dst_labels_dir, file_name.replace(".png", ".txt"))
        shutil.copy(src_label_path, dst_label_path)

# 이미지와 레이블 파일을 train, val, test로 분할
def split_data(base_dir):
    # images와 labels 경로
    images_dir = os.path.join(base_dir, "images")
    labels_dir = os.path.join(base_dir, "labels")

    # 모든 이미지 파일 가져오기
    image_files = glob(os.path.join(images_dir, "*.png"))

    # 이미지 파일 랜덤으로 섞기
    random.shuffle(image_files)

    # 분할할 개수 계산
    num_train = int(len(image_files) * TRAIN_RATIO)
    num_val = int(len(image_files) * VAL_RATIO)

    train_files = image_files[:num_train]
    val_files = image_files[num_train:num_train+num_val]
    test_files = image_files[num_train+num_val:]

    # 데이터 이동
    move_images_and_labels(images_dir, labels_dir, train_images_dir, train_labels_dir, [os.path.basename(f) for f in train_files])
    move_images_and_labels(images_dir, labels_dir, val_images_dir, val_labels_dir, [os.path.basename(f) for f in val_files])
    move_images_and_labels(images_dir, labels_dir, test_images_dir, test_labels_dir, [os.path.basename(f) for f in test_files])

# 실행
split_data(BASE_DIR)


In [ ]:
import os

# 데이터셋 경로 설정
dataset_split_dir = "/content/gdrive/MyDrive/mutsa_2nd_project/3_split_data"
yaml_output_path = os.path.join(dataset_split_dir, "data.yaml")

# 클래스 정의
classes = [
    "bottle_broken", "bottle_contamination", "capsule_crack", "capsule_faulty_imprint",
    "capsule_poke", "capsule_scratch", "capsule_squeeze", "pill_color", "pill_crack",
    "pill_faulty_imprint", "pill_pill_type", "pill_scratch", "screw_manipulated_front",
    "screw_thread", "screw_scratch", "tile_crack", "tile_glue_strip", "tile_gray_stroke",
    "tile_oil", "tile_rough", "toothbrush_defective"
]

# YAML 파일 내용 구성
yaml_content = f"""
train: {dataset_split_dir}/train/images
val: {dataset_split_dir}/val/images

# 클래스 수
nc: {len(classes)}
names: {classes}
"""

# YAML 파일 생성
with open(yaml_output_path, "w") as f:
    f.write(yaml_content)

print(f"YAML 파일 생성 완료: {yaml_output_path}")


YAML 파일 생성 완료: /content/gdrive/MyDrive/mutsa_2nd_project/3_split_data/data.yaml


In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 kB 11.2 MB/s eta 0:00:00


Wed Nov 20 01:14:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P0              43W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [5]:
from ultralytics import YOLO

# YOLOv8 모델 로드 (여기서는 YOLOv8s 모델 사용)
model = YOLO('yolov8m-seg.pt')  # 미리 학습된 YOLOv8 세그멘테이션 모델

# 모델 학습
model.train(
    data='/content/gdrive/MyDrive/mutsa_2nd_project/3_split_data/data.yaml',  # YAML 파일 경로
    epochs=200,  # 학습 에폭수
    batch=32,  # 배치 사이즈
    imgsz=864,  # 이미지 사이즈 (img_size 대신 imgsz)
    device=0,  # GPU 디바이스 (0은 첫 번째 GPU 사용)
    name='yolov8_seg_training',  # 모델 저장 폴더 이름
    task = 'segment',
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 52.4M/52.4M [00:00<00:00, 113MB/s]


Ultralytics 8.3.34 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=segment, mode=train, model=yolov8m-seg.pt, data=/content/gdrive/MyDrive/mutsa_2nd_project/3_split_data/data.yaml, epochs=200, time=None, patience=100, batch=32, imgsz=864, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=yolov8_seg_training, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=

100%|██████████| 755k/755k [00:00<00:00, 14.6MB/s]


Overriding model.yaml nc=80 with nc=21

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytic

...

                   all         55         55      0.662      0.678      0.695      0.469      0.417      0.358       0.29     0.0739



200 epochs completed in 0.332 hours.
Optimizer stripped from runs/segment/yolov8_seg_training/weights/last.pt, 54.9MB
Optimizer stripped from runs/segment/yolov8_seg_training/weights/best.pt, 54.9MB

Validating runs/segment/yolov8_seg_training/weights/best.pt...
Ultralytics 8.3.34 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
YOLOv8m-seg summary (fused): 245 layers, 27,234,543 parameters, 0 gradients, 110.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.08it/s]


                   all         55         55      0.757      0.668      0.746      0.488      0.496       0.28      0.281     0.0801
  bottle_contamination          5          5          1      0.898      0.995      0.637      0.941        0.8      0.821      0.391
         capsule_crack          3          3      0.414      0.494      0.497      0.289          0          0     0.0944    0.00944
capsule_faulty_imprint          3          3      0.902          1      0.995      0.736      0.636      0.667      0.471     0.0686
          capsule_poke          2          2          0          0      0.124     0.0638          1          0          0          0
       capsule_scratch          5          5          1      0.856      0.995      0.754      0.547        0.4      0.531      0.131
       capsule_squeeze          2          2      0.752        0.5       0.58      0.522      0.837        0.5       0.58      0.174
            pill_color          5          5          1      0.917   

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 12, 13, 14, 15, 20])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7db094e4ccd0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037